In [1]:
import os
import sys
sys.path.append(os.path.dirname(sys.path[0]))

from birdmaps.habitats import RedList, HabitatGenerator, reproject_shapefile
from birdmaps.scgt import GeoTiff

c:\Users\lawta\anaconda3\envs\bird-mapping\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
resolution = 300
resampling = "near"
crs = 'PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

padding = 200000
bounds = (-2361583.2255652365274727, 1242360.4432584622409195, -1646660.4594384517986327, 2455525.7292816736735404)

refine_method = "forest_add308"
reproject_inputs = False

In [3]:
species_list_path = "./inputs/species_list.txt"
terrain_path = "./inputs/iucn_habclass_lvl2_us.tif"
terrain_codes_path = "./inputs/iucn_habclass_lvl2_us_mapcodes.csv"
output_folder = "./outputs"

In [4]:
# Some file checks to catch bad parameters early on.

assert os.path.isfile(terrain_path), "invalid terrain_path"
assert os.path.isfile(species_list_path), "invalid species_list_path"
if not reproject_inputs:
    assert os.path.isfile(terrain_codes_path), "terrain_codes_path does not exist; set reproject_inputs to True to generate"
else:
    assert os.path.isdir(os.path.dirname(terrain_codes_path)), "invalid terrain_codes_path"
assert resolution == None or isinstance(resolution, int), "invalid resolution"
assert resampling in ["near", "bilinear", "cubic", "cubicspline", "lanczos", "average", "rms", "mode", "max", "min", "med", "q1", "q3", "sum"], "invalid resampling"
assert os.path.isdir(output_folder), "invalid output_folder"

In [5]:
# Get the list of bird species from species_list_path
with open(species_list_path) as file:
    species_list = file.read().splitlines()

terrain_path = os.path.abspath(terrain_path)
terrain_codes_path = os.path.abspath(terrain_codes_path)

# Generate species output folders
for species in species_list:
    species_output_folder = os.path.join(output_folder, species)
    if not os.path.exists(species_output_folder):
        os.makedirs(species_output_folder)

# Specify eBird range file locations
species_range_folder = "./inputs/ebird_ranges"
indiv_range_path = "./inputs/ebird_ranges/2020/{code}/ranges/{code}_range_smooth_mr_2020.gpkg"
indiv_range_layer = "range"

# Create species output folder if it doesn't currently exist
if not os.path.exists(species_output_folder):
    os.makedirs(species_output_folder)

In [6]:
redlist = RedList()
habitat_generator = HabitatGenerator(
    terrain_path=terrain_path,
    terrain_codes_path=terrain_codes_path,
    crs=crs,
    resolution=resolution,
    resampling=resampling,
)

In [7]:
# If specified, reproject and crop the input terrain.

if reproject_inputs:
    habitat_generator.reproject_terrain()
    # Crop terrain if bounds are specified
    if bounds is not None:
        habitat_generator.crop_terrain(bounds=bounds, padding=padding)
    habitat_generator.write_map_codes()
    # Once it's done once, don't do it again
    reproject_inputs = False

In [8]:
# Obtain species habitat information from the IUCN Red List.

species_data = []

for species in species_list:
    sci_name = redlist.get_scientific_name(species)
    habs = redlist.get_habitats(sci_name)

    # When eBird's scientific name differs from that on Red List, we manually correct it here
    if species == "whhwoo":
        sci_name = "Leuconotopicus albolarvatus"
        habs = redlist.get_habitats(sci_name)
    if species == "yebmag":
        sci_name = "Pica nutalli"
        habs = redlist.get_habitats(sci_name)

    if len(habs) == 0:
        print("Skipping", species, "due to not finding info on Red List (perhaps a name mismatch with eBird)?")
        continue
    
    data = {
        "name": species,
        "sci_name": sci_name,
        "habitats": habs
    }
    species_data.append(data)

In [9]:
# Download or extract species ranges as shapefiles from eBird or IUCN, depending on the specified range source.

habitat_generator.get_ranges_from_ebird(species_list_path, species_range_folder)

In [10]:
# Create the resistance table for each species.

all_map_codes = habitat_generator.get_map_codes()
for species in species_data:
    code = species["name"]
    resistance_output_path = os.path.join(output_folder, code, "resistance.csv")
    habitat_generator.generate_resistance_table(species["habitats"], all_map_codes, resistance_output_path)

In [11]:
# Perform the intersection between the range and habitable terrain.

with GeoTiff.from_file(habitat_generator.terrain_path) as ter:
    # In case resolution was initially marked as None
    resolution = int(ter.dataset.transform[0])
    
    for i, species in enumerate(species_data):
        if not os.path.isfile(indiv_range_path.format(code=species["name"])):
            print("{num}. Skipping {code}, no associated indiv_range_path found".format(num=i+1, code=species["name"]))
            continue

        print("{num}. Refining {code}".format(num=i+1, code=species["name"]))

        code = species["name"]

        # Load the range shapes for the given species, and put them into the correct CRS
        range_shapes = reproject_shapefile(
            shapes_path=indiv_range_path.format(code=code),
            dest_crs=crs,
            shapes_layer=indiv_range_layer
        )

        # Depending on the length of range_shapes, determine if this is a seasonal bird.
        if len(range_shapes) == 1:
            # Not a seasonal bird
            path = os.path.join(output_folder, code, f"habitat_2020_{resolution}_{resampling}_{refine_method}.tif")
            habitat_generator.refine_habitat(ter=ter, species=species, shapes=range_shapes[0], output_path=path, refine_method=refine_method)
        else:
            # Seasonal bird, different output for each shape
            for s in range_shapes:
                season = str(s["properties"]["season"])
                path = os.path.join(output_folder, code, f"{season}_habitat_2020_{resolution}_{resampling}_{refine_method}.tif")
                habitat_generator.refine_habitat(ter=ter, species=species, shapes=s, output_path=path, refine_method=refine_method)

1. Refining acowoo
2. Refining oaktit
